In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from convert_pmt_ids import *

In [ ]:
vm = input()
if vm.lower()=='yes':
    path='/mnt/cephfs/ml_data/mc_2021/'
else:
    path=''

yes


In [ ]:
option = input('Enter: AmC/Cs137/Ge68: ', )

In [ ]:
number_of_files = int(input('Enter number of files... ', ))

In [ ]:
data = np.load('{}calib/{}/data/raw_data_train_0.npz'.format(path, option), allow_pickle=True)['a']

for i in range(1, number_of_files):
    data = np.append(
        data,
        np.load('{}calib/{}/data/raw_data_train_{}.npz'.format(path, option, i), allow_pickle=True)['a'],
        axis=1
    )

In [ ]:
NEvents = data.shape[1]
for i in tqdm(range(NEvents)):
    data[0, :][i] = convert_pmt_ids(data[0, :][i])

In [ ]:
PMTPos_CD_LPMT = pd.read_csv('PMTPos_CD_LPMT.csv')
PMTPos_CD_LPMT

# Event visualization

In [ ]:
from visualize_event import visualize_events
data_true = pd.read_csv('{}calib/{}/targets/targets_train_0.csv'.format(path, option))
for i in range(1, number_of_files):
    data_true = data_true.append(
        pd.read_csv('{}calib/{}/targets/targets_train_{}.csv'.format(path, option, i))
    )
data_true = data_true.reset_index().drop('index', axis=1)

data_true['edepR'] = (data_true['edepX']**2 + data_true['edepY']**2 + data_true['edepZ']**2)**0.5

random_evts = np.random.randint(0, len(data_true), 5)
R_array = np.array(PMTPos_CD_LPMT['R'][random_evts]) / 1000.

lpmt_x_array = []
lpmt_y_array = []
lpmt_z_array = []

for i in range(len(random_evts)):
    lpmt_x = np.array(PMTPos_CD_LPMT['x'][data[0, :][random_evts[i]]]) / 1000.
    lpmt_y = np.array(PMTPos_CD_LPMT['y'][data[0, :][random_evts[i]]]) / 1000.
    lpmt_z = np.array(PMTPos_CD_LPMT['z'][data[0, :][random_evts[i]]]) / 1000.

    lpmt_x_array.append(lpmt_x)
    lpmt_y_array.append(lpmt_y)
    lpmt_z_array.append(lpmt_z)

lpmt_charge_array = data[1, :][random_evts]
lpmt_fht_array = data[2, :][random_evts]

event_edep_array = np.array(data_true['edep'][random_evts])
event_Redep_array = np.array(data_true['edepR'][random_evts])
event_pos_x_array = np.array(data_true['edepX'][random_evts])
event_pos_y_array = np.array(data_true['edepY'][random_evts])
event_pos_z_array = np.array(data_true['edepZ'][random_evts])
                      
visualize_events(
    random_evts,
    R_array.mean(),
    lpmt_x_array,
    lpmt_y_array,
    lpmt_z_array,
    lpmt_charge_array,
    lpmt_fht_array,
    event_pos_x_array,
    event_pos_y_array,
    event_pos_z_array,
    event_edep_array,
    event_Redep_array,
    subplots=True,
    colorscale_fht='ice',
    cc_show=True,
    cht_show=True
)

In [ ]:
import plotly.graph_objects as go
import plotly.io as pio
pio.templates.default = 'plotly_white'

def plot_cdf_pdf(
    evts,
    data,
    colors,
    mode='nPE',
    base='R',
    xbins=dict(
        start=0,
        end=30,
        size=0.2
    ),
    x_dtick=5,
    y1_dtick=0.25,
    y2_dtick=0.05,
    y_domain=[0.08, 0.8],
    x_domain=[0.3, 0.95]
):

    if mode=='nPE':
        ind = 1
        x_title='nPE'
    elif mode=='FHT':
        ind = 2
        x_title='t, ns'
        
    if base=='R':
        name = lambda x: "R = " + str(x.round(2)) + ' m'
    elif base=='E':
        name = lambda x: "E = " + str(x.round(2)) + ' MeV'
        
    fig = go.Figure()

    for i in range(len(evts)):
        line = sorted(data[ind, :][evts[i]])
        hist = data[ind, :][evts[i]]
        if base=='R':
            true = data_true['edepR'][evts[i]]
        elif base=='E':
            true = data_true['edep'][evts[i]]
        
        fig.add_trace(
            go.Scatter(
                y=np.arange(len(line))/len(line),
                x=line,
                name=name(true), #"R = " + str(true.round(2)) + ' m',
                marker_color=colors[i],
                opacity=0.75,
            )
        )

        fig.add_vrect(
                x0=np.mean(hist),
                x1=np.mean(hist),
                line_width=1.2,
                line_color=colors[i],
                line_dash="dash",
                opacity=0.75,
        )
        if i % 2 == 0:
            fig.add_trace(
                go.Histogram(
                    x=hist,
                    marker_color=colors[i],
                    name=name(true),#"R = " + str(true.round(2)) + ' m',
                    showlegend=False,
                    opacity=0.6,
                    histnorm='probability',
                    xaxis='x2',
                    yaxis='y2',
                    xbins=xbins
                )
            )

    fig.update_layout(
        xaxis_title=x_title,
        yaxis_title="F(t)",
        barmode='overlay',
#         title='Deposited energy: {} MeV'.format(data_true['edep'][evts[0]].round(5)),
        legend=dict(
                orientation="h",
                yanchor="bottom",
                y=1.05,
                xanchor="right",
                x=1
        ),
        xaxis1 = dict(
            showline=True,
            ticks='outside',
            mirror=True,
            linecolor='black',
            showgrid=True,
            tick0=0,
            dtick=x_dtick,
            gridcolor='grey',
            gridwidth=0.25,
        ),
        yaxis1 = dict(
            showline=True,
            ticks='outside',
            mirror=True,
            linecolor='black',
            tickmode='linear',
            tick0=0,
            dtick=y1_dtick,
            showgrid=True,
            gridcolor='grey',
            gridwidth=0.25,
        ),
        xaxis2 = dict(
            showline=True,
            ticks='outside',
            mirror=True,
            linecolor='black',
            showgrid=True,
            tick0=0,
            dtick=x_dtick,
            gridcolor='grey',
            gridwidth=0.25,
            domain=x_domain,
            anchor='y2'
        ),
        yaxis2 = dict(
            showline=True,
            ticks='outside',
            mirror=True,
            linecolor='black',
            tickmode='linear',
            tick0=0,
            dtick=y2_dtick,
            showgrid=True,
            gridcolor='grey',
            gridwidth=0.25,
            domain=y_domain,
            anchor='x2'
        ),
        
        font = dict(
            family="Times New Roman",
            size=16,
            color='black'
        )
    )

    fig.show()
    pio.write_image(fig, 'plots/calib/{}/plot_cdf_pdf_{}_{}.pdf'.format(option, mode, base), width=950, height=600)

In [ ]:
if option == "Ge68":
    evts = [498, 2, 13, 12, 0]
elif option == "Cs137":
    evts = [136, 483, 596, 688, 949]
else:
    evts = np.random.randint(0, len(data_true), 5)

In [ ]:
print("Event Ids:", *evts)
colors = ['darkred', 'royalblue', 'green', 'purple', 'gold']

plot_cdf_pdf(evts, data, colors, base='E')

In [ ]:
print("Event Ids:", *evts)
xbins = dict(start=0, end=1200, size=10)
x_domain = [0.45, 0.95]
y_domain = [0.1, 0.7]

plot_cdf_pdf(
    evts, data, colors, 
    mode='FHT', base='E',
    xbins=xbins, x_dtick=200,
    y1_dtick=0.25, x_domain=x_domain,
    y2_dtick=0.05, y_domain=y_domain
)

In [ ]:
fig = go.Figure()

fig.add_trace(
    go.Histogram(
        x=data_true['edep'],
        histnorm='probability'
    )
)

fig.update_layout(
    title="Dep. energy distr. for {}".format(option),
    xaxis_title="Edep, MeV",
    font = dict(
        family="Times New Roman",
        size=16,
        color='black'
    )
)

fig.show()

In [ ]:
fig = go.Figure()

fig.add_trace(
    go.Histogram(
        x=data_true['edepR'],
        histnorm='probability'
    )
)

fig.update_layout(
    title="R distr. for {}".format(option),
    xaxis_title="R, m",
    font = dict(
        family="Times New Roman",
        size=16,
        color='black'
    )
)

fig.show()